### About 

The purpose of this notebook is to experiment & have a working training workflow:

* make changes to data 
* make TFRecordDataset
* get model weights and configuration file
* change configuration file
* train and pray 



### Loading modules & importing dataset 

In [ ]:
# Install modules
!pip install -q Cython contextlib2 pycocotools tf_slim
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

# Used for rotation
!pip install -q -U tensorflow-addons

     |████████████████████████████████| 358kB 5.5MB/s 
Selecting previously unselected package python-bs4.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models:/content/models/research/:/content/models/research/slim/'
import sys
sys.path.append("/content/models")

!python object_detection/builders/model_builder_test.py

/content
fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research
2021-01-24 11:10:45.806544: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import shutil
import json
import random
import re
import io
import imageio
import cv2
import glob
import scipy.misc
import math
import numpy as np
import tensorflow_addons as tfa
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

import tensorflow_hub as hub

%matplotlib inline

In [ ]:
# 'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
# 'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
# 'batch_size': 12

#/content/models/research/object_detection/configs/tf2/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.config
# /content/models/research/object_detection/configs/tf2/faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.config

### Data

In [ ]:
%cd /content/data
!curl -L "https://app.roboflow.com/ds/PzLODKbsNY?key=Zu2TglfXnL" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content/data
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   891  100   891    0     0   1323      0 --:--:-- --:--:-- --:--:--  1321
100 17.5M  100 17.5M    0     0  17.7M      0 --:--:-- --:--:-- --:--:-- 17.7M
Archive:  roboflow.zip
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/wastes.tfrecord    
 extracting: test/wastes_label_map.pbtxt  
   creating: train/
 extracting: train/wastes.tfrecord   
 extracting: train/wastes_label_map.pbtxt  
   creating: valid/
 extracting: valid/wastes.tfrecord   
 extracting: valid/wastes_label_map.pbtxt  


In [ ]:
test_record_fname = '/content/data/valid/wastes.tfrecord'
train_record_fname = '/content/data/train/wastes.tfrecord'
label_map_pbtxt_fname = '/content/data/train/wastes_label_map.pbtxt'

In [119]:
MODEL = 'faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8'
pipeline_file = 'faster_rcnn_resnet152_coco.config'
batch_size = 12

# /content/models/research/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz

In [121]:
%cd /content/models/research
# /content/drive/MyDrive/waste_detection/Leon/pre_trained_weights/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz
import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

# if not (os.path.exists(MODEL_FILE)):
#     urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
!cp "/content/drive/MyDrive/waste_detection/Leon/pre_trained_weights/faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8.tar.gz" .
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [122]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 20K
drwxr-x---  4 345018 89939 4.0K Jul 11  2020 .
drwxr-xr-x 27 root   root  4.0K Jan 24 12:44 ..
drwxr-x---  2 345018 89939 4.0K Jul 10  2020 checkpoint
-rw-r-----  1 345018 89939 3.7K Jul 11  2020 pipeline.config
drwxr-x---  3 345018 89939 4.0K Jul 10  2020 saved_model


In [123]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/models/research/pretrained_model/model.ckpt'

In [124]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [125]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [126]:
import re
num_steps = int(1000)
num_eval_steps = 50

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {:d}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [127]:
!cat {pipeline_fname}

# Faster R-CNN with Resnet-152 (v1), configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 10
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet152'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated

In [130]:
pipeline_fname

'/content/models/research/object_detection/samples/configs/faster_rcnn_resnet152_coco.config'

### Train

In [ ]:
!pip install lvis

In [131]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path='/content/models/research/object_detection/samples/configs/faster_rcnn_resnet152_coco.config' \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps=1000 \
    --num_eval_steps=50

2021-01-24 12:45:59.458469: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
W0124 12:46:02.642895 139749049919360 model_lib.py:793] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 1000
I0124 12:46:02.643161 139749049919360 config_util.py:552] Maybe overwriting train_steps: 1000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0124 12:46:02.643297 139749049919360 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0124 12:46:02.643395 139749049919360 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0124 12:46:02.643496 139749049919360 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0124 12:46:02.643621 139749049919360 model_lib.py:809] Expected number of evaluation epochs is 1, but instead encountered `

In [ ]:
pipeline_fname

'/content/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config'